# Open dataset and do some plotting!

This notebook will walk through opening the generated `combined.json` with xarray and using xarray + metpy + hvplot to create interactive plots.

First, import the needed libraries:
* Metpy - used for generating projection info from GOES data
* Cartopy - used for projecting/transforming coordinate systems
* fsspec - open remote files

In [ ]:
import metpy
import hvplot.xarray
import matplotlib.pyplot as plt
import fsspec
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature

### Create a filesystem with fsspec, give it info on json and remote files

In [ ]:
# Use this line for the example JSON
# fs = fsspec.filesystem('reference', fo="./example_jsons/combined.json", remote_protocol='s3', remote_options=dict(anon=True), skip_instance_cache=True)

# Use this line for the JSON created in Part 1
fs = fsspec.filesystem('reference', fo="./combined.json", remote_protocol='s3', remote_options=dict(anon=True), skip_instance_cache=True)

### Filesystem mapper can be opened directly with xarray's zarr engine.

Also add `metpy.parse_cf()` to generate needed projection information (will be used later) and change `t` variable's name from "J2000 epoch mid-point between the start and end image scan in seconds" to "Time"

In [ ]:
%%time
ds = xr.open_dataset(fs.get_mapper(""), engine='zarr').metpy.parse_cf()
ds['t'].attrs['long_name'] = 'Time'
ds

### Plot with hvplot
Plotting is quick and easy with hvplot. Just provide `x`, `y`. 

`crs=` is the coordinate reference system that the data is in (GOES projection's 'x' and 'y' in this case) so that hvplot/cartopy know how to interpret your input

`projection=` is the output projection (can be different than input, cartopy will do the transformation). In this case we just want to plot on the GOES projection.

In [ ]:
sst = ds.reset_coords()['SST']
crs = ds['SST'].metpy.cartopy_crs

sst.hvplot.quadmesh(
    'x', 'y', cmap='jet',
    groupby='t', crs = crs,
    projection=crs, project=True,
    coastline=True, rasterize=True, features=['borders'],
    clim=(220, 320), width=500
)

### Plot with matplotlib
Matplotlib/pyplot is a much more widely-supported plotting package. However, it takes longer to plot the entire dataset than hvplot does, so we'll take a subset first.

In [ ]:
import matplotlib.pyplot as plt

Using metpy again, this function takes the GOES x/y projection info and turns it into lat/lon

In [ ]:
ds = ds.metpy.assign_latitude_longitude()
ds

This function finds the x/y range for a given lat/lon range

In [ ]:
def get_xy_from_latlon(ds, lats, lons):
    import numpy as np
    lat1, lat2 = lats
    lon1, lon2 = lons

    lat = ds.latitude.data
    lon = ds.longitude.data
    
    x = ds.x.data
    y = ds.y.data
    
    x,y = np.meshgrid(x,y)
    
    x = x[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)]
    y = y[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)] 
    
    return min(x), max(x), min(y), max(y)

Let's take a subset from from 20N to 60N and -100E to -50E

In [ ]:
lat1, lat2 = 20, 60
lon1, lon2 = -100, -50

x1, x2, y1, y2 = get_xy_from_latlon(ds, (lat1, lat2), (lon1, lon2))
print(x1, x2, y1, y2)

In [ ]:
ds_subset = ds.sel(x=slice(x1, x2), y=slice(y2, y1))

Similar to hvplot, `transform=` are the coordinates that the data is currently in, and `projection=` is the coordinate system we want to plot with.

In this case, we'll plot on an Orthographic projection centered at 30N -75E

In [ ]:
projection = ccrs.Orthographic(-75, 30)
transform = ds_subset.SST.metpy.cartopy_crs

var = ds_subset.isel(t=0)

fig = plt.figure(figsize=(10,10))
ax = plt.subplot(projection=projection)

p = ax.pcolormesh(
    var.x, var.y, var.SST,
    transform=transform, cmap='jet',
    vmin=220, vmax=320
)

plt.colorbar(p, label='SST [K]', orientation='horizontal')

ax.set_title(f"GOES SST\n{var.t.values}")
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.STATES)
ax.set_extent([x1, x2, y2, y1], crs=transform)